# Teste Machine Learning Docket

Requisitos obrigatórios:
> - Através do processamento de texto e/ou imagem, extrair o número de CPF de um documento CNH;
> - Desenvolver o próprio dataset.

Requisitos opicionais:
> - Extrair outras informações padrões da CNH, como por exemplo data de validade, RG e Nome.

In [1]:
import cv2
import pytesseract
from skimage.filters import threshold_local

import re

# referenciar ao arquivo executável tesseract baixado
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

## 1. Reconhecimento Ótico de Caracteres

#### CNH Digitalizada

<img src="Digitalizada.jpeg" width="400">

In [2]:
print(pytesseract.image_to_string('Digitalizada.jpeg'))

 

 

MARIANA ZEFERINO DOS SANTOS

 

VALIDA EM TODO
O TERRITORIO NACIONAL

 

DOG. IDENTIDADE / ORG. EMISSOR / UF —
(458418811 SSP SP
CPF DATA NASCIMENT
[aas. 712 .888- eslGssisrisss),

FILIAGAO

      
    
  

OS SANTOS
PERMISSAO: CAT. HAB--
( me }
+*HABILITAGAO ==,

 
 
    

    

 
     
 
 

    
 

‘VALIDADE:

46/n2/9n92



#### CNH Foto

<img src="Foto.jpeg" width="400">

In [3]:
pytesseract.image_to_string('Foto.jpeg')

' \n\nMARIANA ZEFERINO\n\n \n\nDOG. IDENTIDADE / ORG. EMISSOR / UF\n458418811 SSP SP\n\nCPF DATA NASCIMENTO,\n| 426.712.888- 6a][15/12/1994 }\n\nFILIAGAO\n\n  \n  \n\n \n   \n\nEe\ncer\nOs SANTOS\n\nPERMISSAO: ACC: CAT. HAB.-\n/ g | | }\n(Comms (26/03/2073 | nn} — }\n\n   \n\nVALIDA EM TODO\nO TERRITORIO NACIONAL\n\nfon\n\x0c'

### 1.1 Melhorando a leitura das imagens e extraindo o texto

[Link de referência](https://medium.com/stackchain/drivers-license-ocr-cnh-with-python-opencv-and-tesseract-21b0388ae101)

In [4]:
def cleanExtractImage(image):
    
    image = cv2.imread(image)
    V = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    # aplicando operações topHat/blackHat
    topHat = cv2.morphologyEx(V, cv2.MORPH_TOPHAT, kernel)
    blackHat = cv2.morphologyEx(V, cv2.MORPH_BLACKHAT, kernel)

    # adicionando e subtraindo entre as operações morfológicas
    add = cv2.add(V, topHat)
    subtract = cv2.subtract(add, blackHat)

    T = threshold_local(subtract, 29, offset=35, method="gaussian", mode="mirror")
    thresh = (subtract > T).astype("uint8") * 255
    
    thresh = cv2.bitwise_not(thresh)
    
    # extraindo texto
    texto = pytesseract.image_to_string(thresh).replace('. ','.').replace('[', '@').replace('(', '@')
    
    return texto

In [5]:
cleanExtractImage('Digitalizada.jpeg')

' \n\n \n\nND = .=\n@MARIANA ZEFERINO DOS SANTOS\n‘ .- DOG.IDENTIDADE / ORG.EMISSOR | UF:\n458418811.SSP SP\n\n<= CPF\nwo | 426.712.888-62] 15/12/2199\n\nFILIAGAO.\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\x0c'

In [6]:
cleanExtractImage('Foto.jpeg')

'  \n       \n\n \n\n    \n\n\\) @MARIANA ZEFERINO DOS SANTOS\né ‘ DOG.IDENTIDADE / ORG, EMISSOR / UF.\noy 458418811 SSP.SP\n\n.CPF BATA NASCIMENTO}\na 426.712.888-62]@15/12/1994 |\n\nFILIAGAO\n\n \n\n   \n  \n  \n\n   \n\nVALIDA EM TODO.\n“OTERRITORIO NACIONAL\n\n2 N REGISTRO\n@oi\n\n&\n\n  \n\n \n\n \n\n \n\x0c'

## 2. Processamento do texto

In [7]:
def getText(image):
    nome = re.findall('@([A-Z ]+)\n', cleanExtractImage(image))
    cpf = re.findall("\d{3}.\d{3}.\d{3}-\d{2}", cleanExtractImage(image))
    rg = re.findall("\d{7,10}", cleanExtractImage(image))
    nascimento = re.findall("\d{2}/\d{2}/\d{4}", cleanExtractImage(image))
    data = {'Nome': nome, 'CPF': cpf, 'RG': rg, 'Nascimento': nascimento}  
    return data

Informações extraídas da CNH Digitalizada:

In [8]:
getText('Digitalizada.jpeg')

{'Nome': ['MARIANA ZEFERINO DOS SANTOS'],
 'CPF': ['426.712.888-62'],
 'RG': ['458418811'],
 'Nascimento': ['15/12/2199']}

Informações extraídas da CNH Foto:

In [9]:
getText('Foto.jpeg')

{'Nome': ['MARIANA ZEFERINO DOS SANTOS'],
 'CPF': ['426.712.888-62'],
 'RG': ['458418811'],
 'Nascimento': ['15/12/1994']}